# MIMIC-FHIR Demo Tutorial
This tutorial will go through the basic of using the mimic-fhir demo data. The main goals will be:
- Import mimic-fhir data to HAPI FHIR server
  - bulk import not implemented on hapi or blaze, need to make bundles which is doable but slow. Just have import as a prequisite.
- Basic $search
- Translate icd9 to icd10 with $translate
- how many patients have X condition
- how many patients have X condition with Y value
- Some basic visualizations

In [ ]:
import requests
import json
import ndjson
import os
import pandas as pd
from pathlib import Path
import time
from dotenv import load_dotenv
#from fhir.resources.parameters import Parameters, ParametersParameter, ParametersParamterType
from py_mimic_fhir.lookup import MIMIC_FHIR_PROFILE_URL
from fhirclient.models.parameters import Parameters, ParametersParameter
from fhirclient.models.searchparameter import SearchParameter

In [ ]:
# Load environment variables
load_dotenv(load_dotenv(Path(Path.cwd()).parents[0] / '.env'))

FHIR_SERVER = os.getenv('FHIR_SERVER')
MIMIC_JSON_PATH = os.getenv('MIMIC_JSON_PATH')
MIMIC_TERMINOLOGY_PATH = os.getenv('MIMIC_TERMINOLOGY_PATH')

## Bulk Import

In [ ]:
parameter_resource = Parameters()
parameter_format = ParametersParameter()
parameter_format.name = 'inputFormat'
parameter_format.valueCode = 'application/fhir+ndjson'

parameter_input = ParametersParameter()
parameter_input.name = 'input'

sub_parameter_type = ParametersParameter()
sub_parameter_type.name = 'type'
sub_parameter_type.valueCode = 'Patient'

sub_parameter_url = ParametersParameter()
sub_parameter_url.name = 'type'
sub_parameter_url.valueUri = f'file://{MIMIC_JSON_PATH}/output_from_hapi/Patient.ndjson'

parameter_input.part = [sub_parameter_type, sub_parameter_url]
parameter_resource.parameter = [parameter_format,parameter_input]

parameter_resource.as_json()


In [ ]:
url = f'{FHIR_SERVER}$import'

resp = requests.post(url, json=parameter_resource.as_json(), headers={"Content-Type": "application/fhir+json"} )

print(resp.json)

## Basic searches

#### Gender breakdown

In [ ]:
search_url = f'{FHIR_SERVER}/Patient?gender=male'
resp_male = requests.get(search_url, headers={"Content-Type": "application/fhir+json"})
search_url = f'{FHIR_SERVER}/Patient?gender=female'
resp_female = requests.get(search_url, headers={"Content-Type": "application/fhir+json"})

print(f"There are {resp_male.json()['total']} male patients and {resp_female.json()['total']} female patients")

#### Gender and birthdate

In [ ]:
search_url = f'{FHIR_SERVER}/Patient?ethnicity=2186-5'
resp_patient = requests.get(search_url, headers={"Content-Type": "application/fhir+json"})
resp_patient.json()

In [ ]:
search_url = f'{FHIR_SERVER}/Patient?gender=female&maritalStatus=W'
resp_patient = requests.get(search_url, headers={"Content-Type": "application/fhir+json"})
resp_patient.json()
# print(f"There are {resp_patient.json()['total']} patients that are female and born before 2075")

## Find condition X

In [ ]:
# look at hypertension with ICD code: I10  (Essential (primary) hypertension)
search_url = f'{FHIR_SERVER}/Condition?code=I10'  #:text=Essential (primary) hypertension'
resp_condition = requests.get(search_url, headers={"Content-Type": "application/fhir+json"}) 
print(f"There were {resp_condition.json()['total']} incidents of essential hypertension in the demo dataset")

## Find condition X with value Y

In [ ]:
# look at lab value equal or higher
fhir_profile = 'http://fhir.mimic.mit.edu/StructureDefinition/mimic-observation-labevents'
search_url = f'{FHIR_SERVER}/Observation?_profile={fhir_profile}&code=50983&_total=accurate' 
resp_observ1 = requests.get(search_url, headers={"Content-Type": "application/fhir+json"}) 
search_url = f'{FHIR_SERVER}/Observation?_profile={fhir_profile}&code=50983&value-quantity:gt=145&_total=accurate'  
resp_observ2 = requests.get(search_url, headers={"Content-Type": "application/fhir+json"}) 

print(f"There were {resp_observ1.json()['total']} sodium readings, with {resp_observ2.json()['total']} appearing above the reference range")

In [ ]:
resp_observ.json()

In [ ]:
# look at lab value equal or higher
fhir_profile = 'http://fhir.mimic.mit.edu/StructureDefinition/mimic-observation-labevents'
search_url = f'{FHIR_SERVER}/Patient?_observation|code=509839999999&_total=accurate' 
resp_pat_obs = requests.get(search_url, headers={"Content-Type": "application/fhir+json"}) 

#### Get interpretation from Observation
- need to create a SearchParameter since it is not present in the base server

In [ ]:
# add interpretation search parameter
search_param = SearchParameter()
search_param.id = 'Observation-interpretation'
search_param.url = 'http://fhir.mimic.mit.edu/SearchParameter/Patient-ethnicity-extension'
search_param.name = 'interpretation'
search_param.status = 'draft'
search_param.type = 'token'
search_param.code = 'interpretation'
search_param.description = 'Interpretation for an observation'
search_param.base = ['Observation']
search_param.expression = "Observation.interpretation.coding.code"
search_param.as_json()

url_search_param = f'{FHIR_SERVER}/SearchParameter/{search_param.id}'
resp = requests.put(url_search_param, json=search_param.as_json(), headers={"Content-Type": "application/fhir+json", "fhirVersion": "4.0"})


#reindex
parameter_reindex = Parameters()
parameter = ParametersParameter()
parameter.name = 'type'
parameter.valueString = 'Observation'
parameter_reindex.parameter = [parameter]
url_reindex = f'{FHIR_SERVER}$mark-all-resources-for-reindexing'
resp_reindex = requests.post(url_reindex, json=parameter_reindex.as_json(), headers={"Content-Type": "application/fhir+json"})
resp_reindex.json()

In [ ]:
resp2.json()

In [ ]:
# look at patient ethnicity extension
search_url = f'{FHIR_SERVER}Observation?interpretation=abnormal&_total=accurate' 
resp_pat = requests.get(search_url, headers={"Content-Type": "application/fhir+json"}) 
resp_pat.json()

## Get extension values

In [ ]:
# add birthsex search parameter
search_param = SearchParameter()
search_param.id = 'Patient-birthsex'
search_param.url = 'http://fhir.mimic.mit.edu/SearchParameter/Patient-birthsex-extension'
search_param.name = 'birthsex'
search_param.status = 'draft'
search_param.type = 'token'
search_param.code = 'birthsex'
search_param.description = 'Birthsex extension from US Core'
search_param.base = ['Patient']
search_param.expression = "Patient.extension.where(url='http://hl7.org/fhir/us/core/StructureDefinition/us-core-birthsex').value"
search_param.as_json()

In [ ]:
# add ethnicity search parameter
search_param = SearchParameter()
search_param.id = 'Patient-ethnicity-extension'
search_param.url = 'http://fhir.mimic.mit.edu/SearchParameter/Patient-ethnicity-extension'
search_param.name = 'ethnicity'
search_param.status = 'draft'
search_param.type = 'token'
search_param.code = 'ethnicity'
search_param.description = 'Ethnicity extension from US Core'
search_param.base = ['Patient']
search_param.expression = "Patient.extension.where(url='http://hl7.org/fhir/us/core/StructureDefinition/us-core-ethnicity').extension.where(url='text').value"
search_param.as_json()

In [ ]:
#create search parameter
url_search_param = f'{FHIR_SERVER}/SearchParameter/{search_param.id}'
resp = requests.put(url_search_param, json=search_param.as_json(), headers={"Content-Type": "application/fhir+json"})

#reindex
parameter_reindex = Parameters()
parameter = ParametersParameter()
parameter.name = 'type'
parameter.valueString = 'Patient'
parameter_reindex.parameter = [parameter]
url_reindex = f'{FHIR_SERVER}$mark-all-resources-for-reindexing'
resp_reindex = requests.post(url_reindex, json=parameter_reindex.as_json(), headers={"Content-Type": "application/fhir+json"})
resp_reindex.json()

In [ ]:
# look at patient ethnicity extension
search_url = f'{FHIR_SERVER}Patient?ethnicity=Not Hispanic or Latino&_total=accurate' 
resp_pat = requests.get(search_url, headers={"Content-Type": "application/fhir+json"}) 
resp_pat.json()

In [ ]:
resp_reindex.headers

## Microbiology

In [ ]:
profile_test = MIMIC_FHIR_PROFILE_URL['ObservationMicroTest']
profile_org = MIMIC_FHIR_PROFILE_URL['ObservationMicroOrg']
profile_susc = MIMIC_FHIR_PROFILE_URL['ObservationMicroSusc']

search_url = f'{FHIR_SERVER}Observation?_profile={profile_test}&code:not=90201&_total=accurate' 
resp_micro = requests.get(search_url, headers={"Content-Type": "application/fhir+json"}) 
resp_micro.json()